In [37]:
import math
import numpy as np
import torch.nn as nn
import torch
import cv2
from torchvision import transforms, datasets
from torch import optim 
from torchvision import models
from torchsummary import summary
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import mediapipe as mp
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [41]:
train_tfms = tt.Compose([tt.Grayscale(num_output_channels=1),
                         tt.RandomHorizontalFlip(),           
                         tt.RandomRotation(30),               
                         tt.ToTensor(),
                         tt.Normalize(mean=[0.485],
                                       std=[0.229])
                        ])    

test_tfms = tt.Compose([tt.Grayscale(num_output_channels=1),
                         tt.ToTensor(),
                         tt.Normalize(mean=[0.485],
                                       std=[0.229])
                       ])

In [42]:
train_ds = ImageFolder('C:\\train', train_tfms)
test_ds = ImageFolder('C:\\test', test_tfms)

In [43]:
loader_train = torch.utils.data.DataLoader(dataset=train_ds,
                                           batch_size=8, 
                                           shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset=test_ds, batch_size=8,
                                          shuffle=False)

In [47]:
class Net(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.net = nn.Sequential(
          nn.Conv2d(1, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.Conv2d(64, 64, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(64), 
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(64, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128), 
          nn.Conv2d(128, 128, 3),
          nn.LeakyReLU(),          
          nn.BatchNorm2d(128),                     
          nn.MaxPool2d(2, 2), #16
          nn.Conv2d(128, 256, 3),
          nn.LeakyReLU(),
          nn.BatchNorm2d(256),           
          nn.Conv2d(256, 512, 3),
          nn.Flatten(),
          nn.Linear(512, 1024),
          nn.LeakyReLU(),    
          nn.Dropout(0.2),
          nn.BatchNorm1d(1024),                 
          nn.Linear(1024, 1024),
          nn.LeakyReLU(), 
          nn.Dropout(0.2),             
          nn.BatchNorm1d(1024),                    
          nn.Linear(1024, 10), 
          nn.Softmax(dim=1)
        )
        self.mean_target_train = 0
        self.mean_target_test = 0
        self.history = None
        self.device=device

        
    def forward(self, x):
        x = self.net(x)
        return x

    
    def fit(self, loader_train, loadre_test, epochs, loss, optimazer, lr, show_log=True):
        self.train()
        self.history = []
        self.optimazer = optimazer(self.parameters(), lr)
        self.loss = loss()
        for epoch in range(epochs):
            running_loss = 0.0
            running_items = 0.0
            predict_ture = 0.0            
            for i, data in enumerate(loader_train):
                X_train, y_train = data[0].to(self.device), data[1].to(self.device)
                self.optimazer.zero_grad()
                outputs = self(X_train)
                loss_iter = self.loss(outputs, y_train)
                loss_iter.backward()
                self.optimazer.step()
                if i%200 ==0:
                    loss_test, acc_test = self.loss_fn(loader_test)
                    self.history.append([epoch+1, i+1, acc_test])
                    if show_log:
                        print(f'Epoch [{epoch + 1:2}/{epochs}]. ' \
                              f'Step [{i + 1:3}/{len(loader_train)}]. ' \
                              f'Loss_test: {loss_test:.4f}  '\
                              f'Acc_test: {acc_test:.3f}'
                              )
                running_loss, running_items = 0.0, 0.0
        print(f'Training is finished with optimazer: {optimazer}')
        return self.history

    
    def loss_fn(self, loader):
        self.eval()
        running_loss = 0.0
        running_items = 0.0
        predict_ture = 0.0        
        for i, data in enumerate(loader):
            X_data, y_data = data[0].to(self.device), data[1].to(self.device)
            outputs = self(X_data)
            loss_iter = self.loss(outputs, y_data)
            running_loss += loss_iter.item()
            _, predicted = torch.max(outputs, 1)
            predict_ture += (y_data==predicted).sum().item()
            running_items += len(y_data)
        self.train()  
        return running_loss / running_items, predict_ture/running_items

In [48]:
net = Net(device).to(device)

In [49]:
summary(net.to(device), input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]             640
         LeakyReLU-2           [-1, 64, 30, 30]               0
       BatchNorm2d-3           [-1, 64, 30, 30]             128
            Conv2d-4           [-1, 64, 28, 28]          36,928
         LeakyReLU-5           [-1, 64, 28, 28]               0
       BatchNorm2d-6           [-1, 64, 28, 28]             128
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 12, 12]          73,856
         LeakyReLU-9          [-1, 128, 12, 12]               0
      BatchNorm2d-10          [-1, 128, 12, 12]             256
           Conv2d-11          [-1, 128, 10, 10]         147,584
        LeakyReLU-12          [-1, 128, 10, 10]               0
      BatchNorm2d-13          [-1, 128, 10, 10]             256
        MaxPool2d-14            [-1, 12

In [50]:
hist = net.fit(loader_train, loader_test, 2, nn.CrossEntropyLoss, optim.Adam, 0.0001, show_log=True)

Epoch [ 1/2]. Step [  1/2000]. Loss_test: 0.2878  Acc_test: 0.100
Epoch [ 1/2]. Step [201/2000]. Loss_test: 0.2491  Acc_test: 0.471
Epoch [ 1/2]. Step [401/2000]. Loss_test: 0.2303  Acc_test: 0.625
Epoch [ 1/2]. Step [601/2000]. Loss_test: 0.2316  Acc_test: 0.618
Epoch [ 1/2]. Step [801/2000]. Loss_test: 0.2198  Acc_test: 0.737
Epoch [ 1/2]. Step [1001/2000]. Loss_test: 0.2157  Acc_test: 0.763
Epoch [ 1/2]. Step [1201/2000]. Loss_test: 0.2178  Acc_test: 0.736
Epoch [ 1/2]. Step [1401/2000]. Loss_test: 0.2124  Acc_test: 0.743
Epoch [ 1/2]. Step [1601/2000]. Loss_test: 0.2107  Acc_test: 0.794
Epoch [ 1/2]. Step [1801/2000]. Loss_test: 0.2078  Acc_test: 0.827
Epoch [ 2/2]. Step [  1/2000]. Loss_test: 0.2238  Acc_test: 0.672
Epoch [ 2/2]. Step [201/2000]. Loss_test: 0.2093  Acc_test: 0.797
Epoch [ 2/2]. Step [401/2000]. Loss_test: 0.2113  Acc_test: 0.782
Epoch [ 2/2]. Step [601/2000]. Loss_test: 0.2149  Acc_test: 0.747
Epoch [ 2/2]. Step [801/2000]. Loss_test: 0.2100  Acc_test: 0.776
Epoch

In [51]:
torch.save(net, './model_1.pth')